In [59]:
import sys,os
sys.path.append("..")

MONGO_URL="localhost:27017"
os.environ["MONGO_URL"] = MONGO_URL

MONGO_INITDB_ROOT_USERNAME="root" # 
os.environ["MONGO_INITDB_ROOT_USERNAME"] = MONGO_INITDB_ROOT_USERNAME

MONGO_INITDB_ROOT_PASSWORD="dr5[gnhn" # 
os.environ["MONGO_INITDB_ROOT_PASSWORD"] = MONGO_INITDB_ROOT_PASSWORD

DATABASE_NAME = "admin"
os.environ["MONGO_INITDB_DATABASE"] = DATABASE_NAME

MONGO_BOT_USERNAME="bot"
os.environ["MONGO_BOT_USERNAME"] = MONGO_BOT_USERNAME

MONGO_BOT_PASSWORD="botpsswd"
os.environ["MONGO_BOT_PASSWORD"] = MONGO_BOT_PASSWORD

from modules.db.dbconfig import root_client

from modules.db.dbschema import UserReagents

from modules.db.users import UsersCollection
users_collection = UsersCollection(db_client, MONGO_VENDORBOT_DATABASE)

user_id = 122267418
query = {"user_id": user_id}

print(root_client)
root_client.list_database_names()

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', connecttimeoutms=5), 'vendorbot_db'), 'users_collection')
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', connecttimeoutms=5)


['admin', 'blacklist_rdkit_db', 'config', 'local', 'test_db', 'vendorbot_db']

In [60]:
users_collection.get_user(user_id)

{'_id': 122267418,
 'user_id': 122267418,
 'username': 'radical_subject',
 'firstname': 'Oleg',
 'lastname': 'Fedorov',
 'phone_number': '',
 'user_reagents': [{'reagent_internal_id': 'a05785ae720b414ca41526baf22bc026',
   'CAS': '2749-11-3',
   'SMILES': 'C[C@H](N)CO',
   'sharing_status': 'shared',
   'timestamp': '18.09.2022 21:11',
   'contact': '+79168685803'},
  {'reagent_internal_id': 'b0a2c693ec494d77ab137b856bcc52c9',
   'CAS': '75-64-9',
   'SMILES': 'CC(C)(C)N',
   'sharing_status': 'shared',
   'timestamp': '18.09.2022 21:11',
   'contact': '+79168685803'}]}

In [18]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [56]:
# SMILES
smiles = 'CC[C@H]([NH3+])CO'



In [58]:
def neutralize_atoms(mol):
    '''
    atom neutralizer
    '''
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

from rdkit.Chem.rdmolops import GetFormalCharge
def charge_fixer(SMILES):
    '''
    Neutralize molecules atom by atom
    '''
    # Create RDKit molecular objects
    mol = Chem.MolFromSmiles(smiles)
    if GetFormalCharge(mol) != 0:
        neutralize_atoms(mol)
    return Chem.MolToSmiles(mol)

In [69]:
from rdkit import RDLogger, Chem
from rdkit.Chem import PandasTools, SanitizeMol

from mongordkit.Database import write
from mongordkit.Search import similarity, substructure
from mongordkit.Database import registration

NameError: name 'MolDoc' is not defined

In [83]:
rdmol = Chem.MolFromSmiles('Cc1ccccc1')
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id', "a05785ae720b414ca41526baf22bc026")
scheme.generate_mol_doc(rdmol)

{'rdmol': Binary(b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x07\x00\x00\x00\x80\x01\x06\x00`\x00\x00\x00\x01\x03\x06@(\x00\x00\x00\x03\x04\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x0b\x00\x01\x00\x01\x02h\x0c\x02\x03h\x0c\x03\x04h\x0c\x04\x05h\x0c\x05\x06h\x0c\x06\x01h\x0c\x14\x01\x06\x01\x06\x05\x04\x03\x02\x17\x04\x00\x00\x00\x00\x00\x00\x00\x16', 0),
 'index': 'YXFVVABEGXRONW-UHFFFAOYSA-N',
 'smiles': 'Cc1ccccc1',
 'scheme': 'default',
 'hashes': {'inchikey_standard': 'YXFVVABEGXRONW-UHFFFAOYSA-N',
  'CanonicalSmiles': 'Cc1ccccc1'},
 'fingerprints': {},
 'value_data': {'reagent_internal_id': 'a05785ae720b414ca41526baf22bc026'}}

In [85]:
write.WriteFromMolList(db_client[MONGO_VENDORBOT_DATABASE].users_collection, [mol], scheme=registration.MolDocScheme()) 

populating mongodb collection with compounds from list...
0 molecules successfully imported
1 duplicates skipped


0

In [102]:
scheme = registration.MolDocScheme()
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id', "a05785ae720b414ca41526baf22bc02")
scheme.generate_mol_doc(rdmol)
# scheme.add_hash_field('reagent_internal_id', )
# scheme.set_index('reagent_internal_id') 
write.WriteFromMolList(db_client[MONGO_VENDORBOT_DATABASE].users_collection, [mol], 
                   scheme)

populating mongodb collection with compounds from list...
0 molecules successfully imported
1 duplicates skipped


0

In [99]:
[mol]

In [49]:
# conda list

In [50]:
# db_instances = dict(root=MongoDriver(root_credentials), timerbot_db=MongoDriver(timerbot_credentials))
from modules.ourbot.handlers.initial import Inital
from modules.db.dbmodel import get_records, get_timerdata_object, purge, add_records

In [51]:
initial = Inital(db_instances)

In [52]:
initial.timerbot_db_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [53]:
add_records(initial.timerbot_db_client, initial.db_instances["timerbot_db"], initial.collection, data)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение, Timeout: 30s, Topology Description: <TopologyDescription id: 623af54055d7015da9bf5d97, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение')>]>

In [ ]:
purge(initial.root_client, initial.db_instances["timerbot_db"])

In [3]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdmolfiles, AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
# for similarity search need mongo-rdkit
# from mongordkit.Search import similarity, substructure, utils
# from mongordkit import Search
# from mongordkit.Database import create, write


def update_rdkit_with_sialdrich (client, db_instance):
    # Disable rdkit warnings
    rdkit.RDLogger.DisableLog('rdApp.*')

    db_name = db_instance.DATABASE_NAME
    molecules_collection = client[db_name]["molecules"]


In [6]:
db_name = db_instance.DATABASE_NAME

NameError: name 'db_instance' is not defined

In [12]:
db_name

'timerbot_db'

In [5]:
client[db_name].molecules

NameError: name 'client' is not defined

In [19]:
client.drop_database("timerbot_db")